In [4]:
import numpy as np
from scipy import misc
import pandas as pd
from itertools import combinations, chain
import json

In [6]:
BLOCK_LOOKUP_FPATH = "data/block_lookup.csv"
block_lookup = pd.DataFrame.from_csv(BLOCK_LOOKUP_FPATH,
                                    header=0, index_col=None)
wells_lookup = pd.DataFrame.from_csv("data/well_table.csv")

In [7]:
sample_json = {
    "_id": {
        "$oid": "592746c6100fd9cccab6ecf6"
    },
    "initiators": [
        "Initiator 1"
    ],
    "monomers": [
        "Monomer 2",
        "Monomer 2",
        "Monomer 3"
    ],
    "terminators": [
        "Terminator 3"
    ],
    "user": "niels",
    "time": 1495746246056
}

In [12]:

def subchain(x):
    vals = choices[x]
    return chain.from_iterable(
        combinations(vals, i) for i in range(1, len(vals)+1))

def define_reactions(inits, monos, terms):
    output = [('','','')]
    for i_set in subchain(inits):   # initiators
        for m_set in subchain(monos):   # monomers
            for t_set in subchain(terms):   # terminators
                
                output.append( (i_set, m_set, t_set))
    return output

In [13]:
min_btype = 0

def subchain_bin(x, c_min=0, c_max=np.float('inf')):
    """subchain_bin:
    
    c_min and c_max are inclusive. x is the set to iterate over
    """
    options = len(x)
    c_max = min(c_max, options)
    return chain.from_iterable(
        combinations(range(options), i) for i in range(c_min, c_max+1))

def fill_form(sub, s_form):
    """Helper for define_reactions_bins"""
    s_entry = s_form.copy()
    if len(sub) > 0:
        s_entry[np.array(sub)] = 1
    return s_entry

def define_reactions_bin(keys_ordered, maxes=None): 
    # init objects
    output = []
    sets = np.array([np.array(choices[c]) for c in keys_ordered])
    forms = np.array([np.zeros(len(s)) for s in sets])
    i_set, m_set, t_set = sets
    i_form, m_form, t_form = forms
    if maxes == None:
        maxes = (1,len(m_set),1)
    else:
        maxes = [len(s) if m < 1 else m for s,m in zip(sets,maxes)]
    i_max, m_max, t_max = maxes
    i_min = 0
    # could also just iterate through set but for consistency and 
    # generalization, i and t options are kept as subchains
    for i_sub in subchain_bin(i_set, i_min, i_max):   # initiators (max 1)
        i_entry = fill_form(i_sub, i_form)
#         this_t_max = t_max if len(i_sub) > 0 else 1
#         this_t_min = min(1, len(i_sub))
        this_t_min, this_t_max = 0,t_max
        for t_sub in subchain_bin(t_set, this_t_min, this_t_max):   # terminators (max 1)
            t_entry = fill_form(t_sub, t_form)
            # 1 if there is/are no initiator and/or terminator, else 0
            this_m_min, this_m_max = (1,1) if min(len(i_sub),len(t_sub))==0 else (0, m_max)
#             this_m_min, this_m_max = 0,m_max
            for m_sub in subchain_bin(m_set, this_m_min, this_m_max):   # monomers
                m_entry = fill_form(m_sub, m_form)
                full_entry = np.array([i_entry, m_entry, t_entry])
                output.append(full_entry)
    output.append(forms.copy())
    output_nparray = np.array(output)
    return output_nparray

#### RULES
1.  may try i, t (no m)
2.  may try no i or no t with ONE m. 
3.  combos of i, m, and t.



In [19]:
choices = sample_json

In [17]:
keys = ['initiators','monomers','terminators']

In [20]:
# key order, named as indiv vars for clarity
maxes = [2,0,2]
combos_arr = define_reactions_bin(keys, maxes)


In [22]:
# build combos equivalence table (can use to export to db)
columns_index = pd.MultiIndex.from_tuples( [(k,b) for k in keys for b in choices[k]], 
                                    names=['category','block_id'])
index_wells = pd.Index(range(0,len(combos_arr)), name='well_id')
combos = pd.DataFrame([np.concatenate(row) for row in combos_arr], 
                         columns=columns_index, index=index_wells, dtype=int)

In [23]:
def write_well_content(row):
    isnull = ""
    values = []
    categories = row.index.levels[0].values
    has_b = 0
    for cat in categories:
#         present = row[cat].where(row[cat]>0).dropna().to_dict().keys()
        non_zero = row[cat].where(row[cat]>0).dropna()
        present = non_zero.reset_index()['block_id'].values
        # ORDERING -- can be changed
        present = sorted(present)
        text = ", ".join(present) if len(present) > 0 else isnull
        has_b += min(1,len(present))
        values.append(text)
    output = pd.Series(values, index=categories)
    return output

# last well is neg control
wells = combos.apply(write_well_content, axis=1).reset_index(drop=True)

In [34]:
wells.values

array([['', 'Monomer 2', ''],
       ['', 'Monomer 2', ''],
       ['', 'Monomer 3', ''],
       ['', 'Monomer 2', 'Terminator 3'],
       ['', 'Monomer 2', 'Terminator 3'],
       ['', 'Monomer 3', 'Terminator 3'],
       ['Initiator 1', 'Monomer 2', ''],
       ['Initiator 1', 'Monomer 2', ''],
       ['Initiator 1', 'Monomer 3', ''],
       ['Initiator 1', '', 'Terminator 3'],
       ['Initiator 1', 'Monomer 2', 'Terminator 3'],
       ['Initiator 1', 'Monomer 2', 'Terminator 3'],
       ['Initiator 1', 'Monomer 3', 'Terminator 3'],
       ['Initiator 1', 'Monomer 2, Monomer 2', 'Terminator 3'],
       ['Initiator 1', 'Monomer 2, Monomer 3', 'Terminator 3'],
       ['Initiator 1', 'Monomer 2, Monomer 3', 'Terminator 3'],
       ['Initiator 1', 'Monomer 2, Monomer 2, Monomer 3', 'Terminator 3'],
       ['', '', '']], dtype=object)

In [24]:
drop_samples = [1,2,4,5,7,8]
wells_mod = wells.drop(drop_samples)
wells_mod.reset_index(inplace=True, drop=True)

In [25]:
wells_mod['sample'] = range(1, len(wells_mod)+1)
# assign wells (this is auto)
wells_mod['well_id'] = range(len(wells_mod)) 
wells_mod['well_name'] = wells_mod.well_id.apply(lambda x: wells_lookup.loc[x,'well_name'])
wells_mod.set_index('well_id',inplace=True)

In [27]:
wells_mod.to_json('wells_listed-idx-well_id.json', orient='index')

In [29]:
wells_mod.to_json()

'{"initiators":{"0":"","1":"","2":"Initiator 1","3":"Initiator 1","4":"Initiator 1","5":"Initiator 1","6":"Initiator 1","7":"Initiator 1","8":"Initiator 1","9":"Initiator 1","10":"Initiator 1","11":""},"monomers":{"0":"Monomer 2","1":"Monomer 2","2":"Monomer 2","3":"","4":"Monomer 2","5":"Monomer 2","6":"Monomer 3","7":"Monomer 2, Monomer 2","8":"Monomer 2, Monomer 3","9":"Monomer 2, Monomer 3","10":"Monomer 2, Monomer 2, Monomer 3","11":""},"terminators":{"0":"","1":"Terminator 3","2":"","3":"Terminator 3","4":"Terminator 3","5":"Terminator 3","6":"Terminator 3","7":"Terminator 3","8":"Terminator 3","9":"Terminator 3","10":"Terminator 3","11":""},"sample":{"0":1,"1":2,"2":3,"3":4,"4":5,"5":6,"6":7,"7":8,"8":9,"9":10,"10":11,"11":12},"well_name":{"0":"A01","1":"A02","2":"A03","3":"A04","4":"A05","5":"A06","6":"A07","7":"A08","8":"A09","9":"A10","10":"A11","11":"A12"}}'

# OTHER THINGS

In [5]:

def check_input_lengths():
    
    for btype in restrictions.keys()+choices.keys():
        error_track = "something else"
        try:
            error_track = "requirements dictionary"
            allowed_b = restrictions[btype]
            error_track = "given choices"
            given_b = len(choices[btype])
            
            if given_b != allowed_b:
                if given_b < 1:
                    print "Didn't get any blocks for %s. Please choose %d blocks."%(btype, allowed_b)
                elif given_b < allowed_b:
                    print "We have only %d blocks for %s. Please choose %d blocks."%(given_b, btype, allowed_b)
                elif given_b > allowed_b:
                    print "%d blocks are too many for %s! Please choose %d blocks."%(given_b, btype, allowed_b)
                return False
        except KeyError as keyerror:
            print "Can't find '%s' in %s."%(btype, error_track)
            return False
        
        
    print "We're good on the number of blocks selected!"
    return True
success = check_input_lengths()

We're good on the number of blocks selected!
